<a href="https://colab.research.google.com/github/componavt/topkar-space/blob/main/wd_compare/compare_two_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from itertools import chain
from difflib import SequenceMatcher
from ctypes import sizeof
import time
!pip install jellyfish
import jellyfish as jf

In [ ]:
#!wget https://github.com/componavt/topkar-space/raw/master/wd_compare/island_query.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/island_query.txt
# cat island_query.txt

!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/islands10.txt

!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/lake_query.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/lakes10.txt
!wget https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/island_q.txt

--2024-04-02 04:03:15--  https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/island_query.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2164 (2.1K) [text/plain]
Saving to: ‘island_query.txt.1’

island_query.txt.1  100%[===================>]   2.11K  --.-KB/s    in 0s      

2024-04-02 04:03:15 (17.3 MB/s) - ‘island_query.txt.1’ saved [2164/2164]

--2024-04-02 04:03:15--  https://raw.githubusercontent.com/componavt/topkar-space/main/wd_compare/islands10.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 

In [ ]:
def find_shared_elements(file1, file2, parametr):
  start_time = time.time()
  f2 = open(file1, 'r', encoding='utf-8')
  f1 = open(file2, 'r',encoding='utf-8')

  file1 = [line.strip().split('	') for line in f1]
  file2 = [line.strip().split('	') for line in f2]


  c,c2 = [],[] #лист для записи данных с Wikidata и Топкар соответственно
  for i in file2:
    for j in file1:
      if SequenceMatcher(None,i[1],j[1]).ratio() > parametr:
        c.append(i)
        c2.append(j)


  final = list(zip(c,c2))#лист с парами данных Wikidata/Топкар
  print(c)
  print(len(c))

  f1.close()
  f2.close()
  print("--- %s seconds ---" % (time.time() - start_time))
  return(final)

['Канзанаволок', 'Кондостров', 'Крестовый', 'Крестовый', 'Святой остров', 'Южный Олений остров', 'Средний', 'Хонкасало', 'Брусно', 'Мягостров', 'Шуйостров', 'Выгостров', 'Кижи', 'Лычный', 'Никольский остров']
15


In [ ]:
from ctypes import sizeof
def find_shared_elements_jf(file1, file2,parametr):
  start_time = time.time()
  f2 = open(file1, 'r', encoding='utf-8')
  f1 = open(file2, 'r',encoding='utf-8')

  file1 = [line.strip().split('	') for line in f1]
  file2 = [line.strip().split('	') for line in f2]


  c,c2 = [],[]#лист для записи данных с Wikidata и Топкар соответственно
  for i in file2:
    for j in file1:
      if jf.levenshtein_distance(i[1],j[1]) < parametr:
        c.append(i)
        c2.append(j)

  final = list(zip(c,c2))#лист с парами данных Wikidata/Топкар
  print(final)

  f1.close()
  f2.close()
  print("--- %s seconds ---" % (time.time() - start_time))
  return(final,c,c2)

In [ ]:
import csv

# c1 = find_shared_elements('island_q.txt', 'islands10.txt', 0.9)
resived,c,c2 = find_shared_elements_jf('island_q.txt', 'islands10.txt', 2)

nodes = c+c2 #Все обекты из Wikidata и Топкар
# print(len(nodes))
sorted_nodes = [list(x) for x in set(tuple(x) for x in nodes)] #Удаляем дубликаты
# print(len(sorted_nodes))
def write_nodes_to_csv(c1, filename):
    with open(filename, 'w', newline='',encoding='utf-8') as csvfile:
        header = ['Id', 'Label']
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)
        for row in c1:
            print(row)
            csvwriter.writerow(row)
        # for row in c1:
        #   print(row)
        #   csvwriter.writerow(list(row[1]))

def write_edges_to_csv(c1, filename):
    with open(filename, 'w', newline='',encoding='utf-8') as csvfile:
        header = ['Source', 'Target']
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(header)
        for row in c1:
            print(row)
            out = [row[0][0],row[1][0]]
            csvwriter.writerow(out)


filename_nodes = 'output_n.csv'
filename_edges = 'output_e.csv'
write_nodes_to_csv(sorted_nodes, filename_nodes) #создаём файл с вершинами гафа
write_edges_to_csv(resived, filename_edges) #создаём файл с рёбрами графа

In [ ]:
c1 = find_shared_elements('island_query.txt', 'islands10.txt')
c2 = find_shared_elements_jf('island_q.txt', 'islands10.txt')

def create_html_table(rows, columns, c2):
    html_table = "<table>"

    for i in range(rows):
        html_table += "<tr>"

        html_table += f"<td><a href ={c2[i][0][0]}> {c2[i][0][0]} <a></td>"
        html_table += f"<td>{c2[i][0][1]}</td>"
        html_table += f"<td>{c2[i][1][0]}</td>"
        html_table += f"<td>{c2[i][1][1]}</td>"
        html_table += "</tr>"

    html_table += "</table>"

    return html_table


def save_html_table_to_file(html_code, file_name):
    with open(file_name, "w") as file:
        file.write(html_code)
    print(f"HTML table saved to {file_name}")



rows = len(c2)
print(f"Rows in toal {rows}")
columns = 4
html_code = create_html_table(rows, columns, c2)
file_name = "table.html"
save_html_table_to_file(html_code, file_name)

{'Крестовый', 'Шуйостров', 'Кижи', 'Южный Олений остров', 'Мягостров', 'Риеккалансаари', 'Лычный', 'Никольский остров', 'Брусно', 'Святой остров', 'Выгостров', 'Хонкасало', 'Средний', 'Канзанаволок', 'Хепосаари', 'Ивановский остров', 'Кондостров'}
17
--- 9.420164346694946 seconds ---
[['http://www.wikidata.org/entity/Q4212426', 'Канзанаволок'], ['http://www.wikidata.org/entity/Q4230532', 'Кондостров'], ['http://www.wikidata.org/entity/Q4230532', 'Кондостров'], ['http://www.wikidata.org/entity/Q4230532', 'Кондостров'], ['http://www.wikidata.org/entity/Q4234186', 'Корписари'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www.wikidata.org/entity/Q4259162', 'Лепостров'], ['http://www